# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233911435116                   -0.50    8.0         
  2   -7.249835417358       -1.80       -1.38    1.0   12.0ms
  3   -7.251061175072       -2.91       -1.87    2.0   14.3ms
  4   -7.251035623181   +   -4.59       -1.91    1.0   11.6ms
  5   -7.251324536804       -3.54       -2.59    1.0   11.5ms
  6   -7.251337607301       -4.88       -3.15    1.0   14.1ms
  7   -7.251338653455       -5.98       -3.52    2.0   17.8ms
  8   -7.251338782571       -6.89       -4.06    1.0   13.8ms
  9   -7.251338796818       -7.85       -4.35    3.0   18.9ms
 10   -7.251338798348       -8.82       -4.80    1.0   14.2ms
 11   -7.251338798668       -9.49       -5.44    3.0   17.7ms
 12   -7.251338798697      -10.53       -5.60    2.0   16.0ms
 13   -7.251338798703      -11.26       -5.98    1.0   13.2ms
 14   -7.251338798704      -11.91       -6.33    2.0   16.0ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.054052209588599145
[ Info: Arnoldi iteration step 2: normres = 0.33964339636786217
[ Info: Arnoldi iteration step 3: normres = 0.3507620012651574
[ Info: Arnoldi iteration step 4: normres = 0.961673847113989
[ Info: Arnoldi iteration step 5: normres = 0.30980174396337945
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.57e-01, 2.85e-02, 1.93e-01, 1.80e-01, 2.62e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5982077185216905
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.20e-02, 6.44e-02, 5.64e-01, 1.29e-01, 1.03e-01)
[ Info: Arnoldi iteration step 7: normres = 0.06851835883642277
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.17e-03, 3.59e-02, 6.95e-03, 3.06e-02, 4.15e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1184371868886506
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.62e-